# Testing file for functions creating the Dataset
Functions in this notebook were moved to static_code folder once they were tested.

In [1]:
import os
import cv2
import torchvision.transforms as transforms
import math

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
normal_dir: str = r'../chest_xray/test/NORMAL'
pneumo_dir: str = r'../chest_xray/test/PNEUMONIA'

assert os.path.exists(normal_dir) and os.path.isdir(normal_dir), "Normal dir isn't found or isn't a directory"
assert os.path.exists(pneumo_dir) and os.path.isdir(pneumo_dir), "Pneumonia dir isn't found or isn't a directory"

In [3]:
def open_preprocess_photos(dir: str, transform, img_size: tuple) -> list:
    img_list = list(map(lambda x: os.path.join(dir, x), os.listdir(dir)))  # Open given directory and list its contents
    img_list.sort()
    images = []  # This list will store the preprocessed images to return

    # Preprocess from pre_process_1.ipynb
    for img in img_list:
        try:
            img_data = cv2.imread(img)
        except Exception:
            print(f"Error occured opening {img} file")
            continue
        shape = img_data.shape
        
        if shape[0] > shape[1]:  # If axis 0 is bigger than axis 1, we crop the image according to axis 0
            diff = shape[0] - shape[1]
            padding = int(math.ceil(diff / 2))
            img_data = img_data[padding : shape[0] - padding, :, :]  # We center the image according to axis 0, to make it square
        else:  # If axis 1 is bigger than axis 0, we crop the image according to axis 1
            diff = shape[1] - shape[0]
            padding = int(math.ceil(diff / 2))
            img_data = img_data[:, padding : shape[1] - padding, :]  # We center the image according to axis 1, to make it square

        img_data = cv2.resize(img_data, img_size)  # Resize image according to given size ((224, 224), for example)
        img_data = transform(img_data)  # Transform the image using the Pytorch transform function given as parameter
        images.append(img_data)
    
    return images

In [4]:
# Basic normalization of image using Pytorch (converts to Tensor and normalizes pixel values)
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

transform = transforms.Compose([
    transforms.ToTensor(),
    normalize,
])

In [9]:
normal_images = open_preprocess_photos(normal_dir, transform, (224, 224))

In [10]:
pneumo_images = open_preprocess_photos(pneumo_dir, transform, (224, 224))

In [11]:
from static_code.Dataset import Dataset

In [12]:
dataset = Dataset(normal=normal_images, pneumonia=pneumo_images, normal_label=0, pneumonia_label=1, batch_size=128)

In [16]:
for (X, y) in dataset:
    print("X:", len(X), "y:", len(y))

X: 128 y: 128
X: 128 y: 128
X: 128 y: 128
X: 128 y: 128
X: 112 y: 112
